In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import os
import time
import calendar
from pathos.multiprocessing import ProcessingPool as Pool
import openpyxl
#from multiprocessing import Pool
from time import sleep

start_time = time.time()
cal = calendar.Calendar()
result_list=[]
new_dict=[]

query = "여성"
year = 2018

def scrape(url):
    import requests
    from bs4 import BeautifulSoup
    try:
        res = requests.get(url, headers={'User-Agent':'Mozilla/5.0'}, allow_redirects=False)
        return res.text
    except:
        return "no text"

if __name__ == '__main__':
    
    for month in range(1, 13):
        monthdays = [d for d in cal.itermonthdays(year, month) if d != 0]
        for day in monthdays:
            s_date = str(year) + "." + str(month).zfill(2) + "." + str(day).zfill(2)
            e_date = s_date 
            s_from = s_date.replace(".","")
            e_to = s_from
            print(s_date)

            xlsx_name = os.path.join(os.path.dirname(os.getcwd()), 'xlsx_data', '#1_'+ str(year) + '_'+query, 
                                     '{}.xlsx'.format("#1_"+s_from+"_"+query))
            wb = openpyxl.Workbook()
            sheet = wb.active
            sheet.append(["title", "source", "date", "html", "url"])
            wb.save(xlsx_name)
            
            book = openpyxl.load_workbook(xlsx_name)
            writer = pd.ExcelWriter(xlsx_name, engine='openpyxl')
            writer.book = book
            writer.sheets = {ws.title: ws for ws in book.worksheets}            

            for n in range(1, 4001, 10):
                url = "https://search.naver.com/search.naver?where=news&query=" + query + "&sort=1&ds=" + s_date + "&de=" + e_date + "&nso=so%3Ar%2Cp%3Afrom" + s_from + "to" + e_to + "%2Ca%3A&start=" + str(n)
                raw = requests.get(url, headers={'User-Agent':'Mozilla/5.0'})
                html = BeautifulSoup(raw.text, "html.parser")
                articles = html.select("ul.type01 > li")

                if not articles:
                    break
                
                title = list()
                urls = list()
                source = list()
                date = list()
                records = list()
                html = list()
                
                for ar in articles:
                    
                    title.append(ar.select_one("a._sp_each_title").text)
                    urls.append(ar.select_one("a").get('href'))
                    source.append(ar.select_one("span._sp_each_source").text)
                    dt = str(ar.select_one("span.bar").next_sibling)
                    if dt:
                        date.append(dt)
                    else:
                        date.append(s_from + ".")
                
                with Pool(10) as p:
                    records = p.map(scrape, urls)
                
                for re in records:
                    html.append(BeautifulSoup(re, "lxml"))
                
                d = pd.DataFrame({"title": title, "source": source, 
                                  "date": date, "html": html, "url": urls}) 
                sleep(0.001)
                
                for sheetname in writer.sheets:
                    try:
                        d.to_excel(writer,sheet_name=sheetname, startrow=writer.sheets[sheetname].max_row, index = False,header= False)
                    except:
                        print("error")
            writer.save()
                
print("--- %s seconds ---" % (time.time() - start_time))

2018.01.01
2018.01.02
2018.01.03


In [ ]:
 raw = requests.get("https://news.naver.com/main/list.nhn?mode=LPOD&mid=sec&oid=032&listType=title&date="+str(date)+"&page="+str(page),
                           headers={"User-Agent":"Mozilla/5.0"})

In [2]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import os
import time
import calendar
from pathos.multiprocessing import ProcessingPool as Pool
import openpyxl
#from multiprocessing import Pool
from time import sleep

start_time = time.time()

result_list=[]
new_dict=[]

query = "여성"
year = 2018

def scrape(url):
    import requests
    from bs4 import BeautifulSoup
    try:
        res = requests.get(url, headers={'User-Agent':'Mozilla/5.0'}, allow_redirects=False)
        return res.text
    except:
        return "no text"

if __name__ == '__main__':
    
    month = 2
    day = 28
    
    s_date = str(year) + "." + str(month).zfill(2) + "." + str(day).zfill(2)
    e_date = s_date 
    s_from = s_date.replace(".","")
    e_to = s_from
    print(s_date)

    xlsx_name = os.path.join(os.path.dirname(os.getcwd()), 'xlsx_data', '#1_'+str(year)+'_'+query, 
                             '{}.xlsx'.format("#1_"+s_from+"_"+query))
    wb = openpyxl.Workbook()
    sheet = wb.active
    sheet.append(["title", "source", "date", "html"])
    wb.save(xlsx_name)

    book = openpyxl.load_workbook(xlsx_name)
    writer = pd.ExcelWriter(xlsx_name, engine='openpyxl')
    writer.book = book
    writer.sheets = {ws.title: ws for ws in book.worksheets}            

    for n in range(1, 4001, 10):
        url = "https://search.naver.com/search.naver?where=news&query=" + query + "&sort=1&ds=" + s_date + "&de=" + e_date + "&nso=so%3Ar%2Cp%3Afrom" + s_from + "to" + e_to + "%2Ca%3A&start=" + str(n)
        raw = requests.get(url, headers={'User-Agent':'Mozilla/5.0'})
        html = BeautifulSoup(raw.text, "html.parser")
        articles = html.select("ul.type01 > li")

        if not articles:
            break
            
        aid = list()
        title = list()
        urls = list()
        source = list()
        date = list()
        records = list()
        html = list()

        for ar in articles:

            aid.append(ar.select_one("a").get('asplugin-id'))
            title.append(ar.select_one("a._sp_each_title").text)
            urls.append(ar.select_one("a").get('href'))
            source.append(ar.select_one("span._sp_each_source").text)
            dt = str(ar.select_one("span.bar").next_sibling)
            if dt:
                date.append(dt)
            else:
                date.append(s_from + ".")

        with Pool(10) as p:
            records = p.map(scrape, urls)

        for re in records:
            html.append(BeautifulSoup(re, "lxml"))

        d = pd.DataFrame({"aid": aid, "title": title, "source": source, 
                          "date": date, "html": html, "url": urls}) 
        sleep(0.001)

        for sheetname in writer.sheets:
            try:
                d.to_excel(writer,sheet_name=sheetname, startrow=writer.sheets[sheetname].max_row, index = False,header= False)
            except:
                print("error")
    writer.save()

print("--- %s seconds ---" % (time.time() - start_time))

2019.02.28
--- 1075.2495594024658 seconds ---
